# Object: Sensor Observation
## Goal
This example shows how to use Observation with sensors

<img src="https://loco-philippe.github.io/ES/sensor.png" width="600">


The Payload is the set of data sent from the sensor to the server.

In [1]:
# This cell is only to prepare the example
import json
from observation import Observation as Obs
from observation import Ilist, NamedValue, LocationValue, DatationValue
from datetime import datetime
from pprint import pprint
import requests as rq
url = "https://webhooks.mongodb-realm.com/api/client/v2.0/app/observation_app-wsjge/service/postObs/incoming_webhook/api?secret=10minutes"

## Case 1: Simple sensor

In this case there is no initialization phase. 

The sensor encode and send an Observation.

The server decode the Observation and store it into a database.

In [2]:
# simulation of a measure
time = "2021-05-05T10:08"
coord = [[2.35, 48.87]]
prop = {"prp":"Temp"}
res = 25.0

# Observation creation and encoding to Json or to binary data in the sensor
ob_sensor = Obs.std(res, time, coord, prop)
payload1 = ob_sensor.json(encoded=True)                         # if the payload is character payload
print(payload1, '\n\ntext payload (length) : ', len(payload1))
payload2 = ob_sensor.json(encoded=True, encode_format='cbor')   # if the payload is binary payload
print('binary payload (length) : ', len(payload2), '\n')                     # 99 bytes
# next : send the payload with the network protocol

# data decoding in the server
ob_receive1 = Obs.obj(payload1)
ob_receive2 = Obs.obj(payload2)
print('data is the same ? ', ob_receive2 == ob_receive1 == ob_sensor, '\n')   # it's True !!

# and store it in the database (example with NoSQL DataBase)
jsonStore = ob_receive1.json(encoded=True)                     # add 'infos' to facilitate the research in the database
pprint(json.loads(jsonStore))
r = rq.post(url, data=jsonStore)
print("\nreponse : ", r.status_code)                           # 200 : Json is stored in the Database


{"type": "observation", "data": [["result", [25.0]], ["datation", ["2021-05-05T10:08:00+00:00"]], ["location", [[2.35, 48.87]]], ["property", [{"prp": "Temp"}]]]} 

text payload (length) :  162
binary payload (length) :  104 

data is the same ?  True 

{'data': [['result', [25.0]],
          ['datation', ['2021-05-05T10:08:00+00:00']],
          ['location', [[2.35, 48.87]]],
          ['property', [{'prp': 'Temp'}]]],
 'type': 'observation'}

reponse :  200


## Case 2: Mobile sensor with one property
In this second example, the sensor is mobile and the property is fixed.

There is no initialization phase.

The sensor encode and send an Observation.

The server decode the Observation

In [3]:
# Observation creation and encoding to Json or to binary data in the sensor
ob_sensor = Obs.std()
prop1 = {'prp': 'PM25', 'unit': 'kg/m3'}
for i in range(6):                                          # simulation of a sequence of measurements
    ob_sensor.append([ 45 + i, datetime(2021, 6, 4+i, 12, 5), [1.1+i, 40.2+i], prop1])
payload2 = ob_sensor.json(encoded=True, encode_format='cbor')
print('binary payload (length) : ', len(payload2), '\n')    # 41.8 bytes/measure
# next : send the payload with the network protocol

# data decoding in the server
ob_receive = Obs.obj(payload2)
print('data is the same ? ', ob_receive == ob_sensor, '\n') # it's True !!

# and store it in the database (example with NoSQL DataBase)
jsonStore = ob_receive.json(encoded=True, json_info=True)   # add 'infos' to facilitate the research in the database
pprint(json.loads(jsonStore), width=120)

binary payload (length) :  251 

data is the same ?  True 

{'data': [['result', [46, 49, 47, 45, 50, 48], [3, 0, 2, 5, 1, 4]],
          ['datation',
           ['2021-06-05T12:05:00+00:00',
            '2021-06-08T12:05:00+00:00',
            '2021-06-06T12:05:00+00:00',
            '2021-06-04T12:05:00+00:00',
            '2021-06-09T12:05:00+00:00',
            '2021-06-07T12:05:00+00:00'],
           0],
          ['location', [[2.1, 41.2], [5.1, 44.2], [3.1, 42.2], [1.1, 40.2], [6.1, 45.2], [4.1, 43.2]], 0],
          ['property', [{'prp': 'PM25', 'unit': 'kg/m3'}]]],
 'information': {'index': {'datation': {'box': ['2021-06-04T12:05:00+00:00', '2021-06-09T12:05:00+00:00'],
                                        'lencodec': 6,
                                        'num': 1,
                                        'typevalue': 'DatationValue'},
                           'location': {'box': [1.1, 40.2, 6.1, 45.2],
                                        'lencodec': 6,
          

## Case 3: Mobile sensor with multiple property
The sensor is mobile and two properties are defined.

There is no initialization phase.

The sensor encode and send an Observation.

The server decode the Observation

In [4]:
# Observation creation and encoding to Json or to binary data in the sensor
ob_sensor = Obs.std()
prop1 = {'prp': 'PM25', 'unit': 'kg/m3'}
prop2 = {'prp': 'PM10', 'unit': 'kg/m3'}
for i in range(6):                                          
    date = datetime(2021, 6, 4 + i, 12, 5)
    loc = [1.1 + i, 40.2 + i]
    ob_sensor.append([ 45 + i, date, loc, prop1])
    if i%3 == 0: ob_sensor.append([ 105 + i//3, date, loc, prop2]) # 2e property is sent with another frequency                      
payload = ob_sensor.full(fillvalue=None).json(encoded=True, encode_format='cbor') # we choose to sent full observation
print('binary payload (length) : ', len(payload2), '\n')    # 243 bytes -> 30.4 bytes/measure
# next : send the payload with the network protocol

# data decoding in the server
ob_receive = Obs.obj(payload)
print('data is the same ? ', ob_receive == ob_sensor, '\n') # it's True !!

pprint(ob_receive.json(json_info=True), width=120)


binary payload (length) :  251 

data is the same ?  True 

{'data': [['result', [106, 46, 105, 49, 47, 45, 50, 48], [5, 2, 1, 4, 7, 0, 3, 6]],
          ['datation',
           [datetime.datetime(2021, 6, 6, 12, 5, tzinfo=datetime.timezone.utc),
            datetime.datetime(2021, 6, 7, 12, 5, tzinfo=datetime.timezone.utc),
            datetime.datetime(2021, 6, 5, 12, 5, tzinfo=datetime.timezone.utc),
            datetime.datetime(2021, 6, 8, 12, 5, tzinfo=datetime.timezone.utc),
            datetime.datetime(2021, 6, 9, 12, 5, tzinfo=datetime.timezone.utc),
            datetime.datetime(2021, 6, 4, 12, 5, tzinfo=datetime.timezone.utc)],
           [0, [1, 2, 5, 3, 0, 5, 4, 1]]],
          ['location', [[3.1, 42.2], [4.1, 43.2], [2.1, 41.2], [5.1, 44.2], [6.1, 45.2], [1.1, 40.2]], 1],
          ['property',
           [{'prp': 'PM25', 'unit': 'kg/m3'}, {'prp': 'PM10', 'unit': 'kg/m3'}],
           [0, [1, 0, 1, 0, 0, 0, 0, 0]]]],
 'information': {'index': {'datation': {'box': [dateti

## Case 4: Simple sensor (minimize data in operation)
This example is the same as the case 1.

In the initialization phase, an Observation with only Property and Location is defined.

In the operation phase, an Observation with only Result is sent.

The complete observation is reconstructed in the server.

In [5]:
# initialization phase (sensor or server) -> once
coord = [2.3, 48.9]
prop = {"prp":"Temp"}
ob_init = Obs.dic({'location': [coord], 'property': prop})
payload_init = ob_init.json()
print('payload_init (in server or sent by sensor) : \n', payload_init)

# operation phase (sensor) -> regularly
res = 25
il_operat = Ilist.obj([res])
#il_operat = Ilist.dic({'res': [res]})
payload1 = il_operat.json(encoded=True)                           # if the payload is character payload
payload2 = il_operat.json(encoded=True, encode_format='cbor')     # if the payload is binary payload
print('\npayload length (text and binary) : ', len(payload1), len(payload2)) # 4 bytes and 3 bytes
# next : send the payload with the network protocol

# data decoding in the server
il_receive1 = Ilist.obj(payload1)
il_receive2 = Ilist.obj(payload2)
date_receive = datetime(2021, 6, 4, 12, 5)
print('\ndata is the same ? ', il_receive1 == il_receive2 == il_operat, '\n')   # it's True !!

# complete observation
ob_complet = Obs.dic({'res': il_receive1, 'datation': date_receive, 'location': [coord], 'property': prop}).merge(simplename=True)
pprint(ob_complet.json(), width=120)


payload_init (in server or sent by sensor) : 
 {'type': 'observation', 'data': [['location', [[2.3, 48.9]]], ['property', [{'prp': 'Temp'}]]]}

payload length (text and binary) :  6 4

data is the same ?  True 

{'data': [['res', [25]],
          ['datation', [datetime.datetime(2021, 6, 4, 12, 5, tzinfo=datetime.timezone.utc)]],
          ['location', [[2.3, 48.9]]],
          ['property', [{'prp': 'Temp'}]]],
 'type': 'observation'}


## Case 5: Sensor with multiple property (minimize data in operation)
In the initialization phase, an Observation with only Property and Location is defined.

In the operation phase, an Observation with only Result is sent.

The complete observation is reconstructed in the server.

In [6]:
# initialization phase (sensor or server) -> once
coord = [2.3, 48.9]
prop1 = {"prp":"PM25"}
prop2 = {"prp":"PM10"}
il_init = Ilist.obj([[coord], prop1, prop2])
payload_init = il_init.json()
print('payload_init (in server or sent by sensor) : \n', payload_init)

# operation phase (sensor) -> regularly
il_sensor = Ilist.dic({'res': [], 'datation':[], 'property':[]})
for i in range(6):                                               # simule une boucle de mesure
    date = datetime(2021, 6, 4+i, 12, 5)
    il_sensor.append([ 45 + i, date, prop1])
    if i%3 == 0: il_sensor.append([ 105 + i//3, date, prop2])
il_sensor.full(fillvalue=None) 
il_sensor.nindex('property').setcodeclist([None, None])
payload = il_sensor.json(encoded=True, encode_format='cbor')
print('\npayload length (binary) : ', len(payload)) # 88 bytes (11 bytes/measure)
# if the full option is not used, the length would be 102 bytes
# next : send the payload with the network protocol

# data decoding in the server
il_init = Ilist.obj(payload_init)
coord = il_init.lindex[0].val[0]
prop1 = il_init.lindex[1].val[0]
prop2 = il_init.lindex[2].val[0]
il_receive = Ilist.obj(payload, reindex=False)
print('\ndata is the same ? ', il_receive == il_sensor, '\n')   # it's True !!
il_receive.nindex('property').setcodeclist([prop1, prop2])

# complete observation
ob_complet = Obs.dic({'res': il_receive, 'location': [coord]}).merge(simplename=True)\
.full(varname=['res'], idxname=['datation', 'property'], inplace=False).setcanonorder()
#.full(idxname=['datation', 'property'], inplace=False).setcanonorder()
pprint(ob_complet.json(fullvar=True), width=120)
print('\n')
ob_complet.view(tablefmt='html', numalign='center', stralign='center')

payload_init (in server or sent by sensor) : 
 [[[2.3, 48.9]], [{'prp': 'PM25'}], [{'prp': 'PM10'}]]

payload length (binary) :  105

data is the same ?  True 

{'data': [['datation',
           [datetime.datetime(2021, 6, 4, 12, 5, tzinfo=datetime.timezone.utc),
            datetime.datetime(2021, 6, 5, 12, 5, tzinfo=datetime.timezone.utc),
            datetime.datetime(2021, 6, 6, 12, 5, tzinfo=datetime.timezone.utc),
            datetime.datetime(2021, 6, 7, 12, 5, tzinfo=datetime.timezone.utc),
            datetime.datetime(2021, 6, 8, 12, 5, tzinfo=datetime.timezone.utc),
            datetime.datetime(2021, 6, 9, 12, 5, tzinfo=datetime.timezone.utc)]],
          ['property', [{'prp': 'PM25'}, {'prp': 'PM10'}]],
          ['res', [45, 105, 46, 47, 48, 106, 49, 50, '-'], [0, 1, 2, 8, 3, 8, 4, 5, 6, 8, 7, 8]],
          ['location',
           [[2.3, 48.9],
            [2.3, 48.9],
            [2.3, 48.9],
            [2.3, 48.9],
            [2.3, 48.9],
            [2.3, 48.9],
   

datation,property,res,location
"""2021-06-04T12:05:00+00:00""","{""prp"": ""PM25""}",45,"[2.3, 48.9]"
"""2021-06-04T12:05:00+00:00""","{""prp"": ""PM10""}",105,"[2.3, 48.9]"
"""2021-06-05T12:05:00+00:00""","{""prp"": ""PM25""}",46,"[2.3, 48.9]"
"""2021-06-05T12:05:00+00:00""","{""prp"": ""PM10""}",-,"""-"""
"""2021-06-06T12:05:00+00:00""","{""prp"": ""PM25""}",47,"[2.3, 48.9]"
"""2021-06-06T12:05:00+00:00""","{""prp"": ""PM10""}",-,"""-"""
"""2021-06-07T12:05:00+00:00""","{""prp"": ""PM25""}",48,"[2.3, 48.9]"
"""2021-06-07T12:05:00+00:00""","{""prp"": ""PM10""}",106,"[2.3, 48.9]"
"""2021-06-08T12:05:00+00:00""","{""prp"": ""PM25""}",49,"[2.3, 48.9]"
"""2021-06-08T12:05:00+00:00""","{""prp"": ""PM10""}",-,"""-"""
